In [ ]:
importing necessary libraries
import pandas as pd
import numpy as np
!pip install pgeocode
import pgeocode
import matplotlib.pyplot as plt
!pip install geopy
from geopy.geocoders import Nominatim
!pip install rapidfuzz
from rapidfuzz import process, fuzz

# Preprossessing Lead Data

In [ ]:
#import first set
current_1 = pd.read_csv('1current_Leads_22028 PM.csv')
dead_1 = pd.read_csv('dead_Leads_1.csv')
warm_1 = pd.read_csv('warm_Leads_21945 PM.csv')

#cobine them
df1 = pd.concat([current_1, dead_1, warm_1], ignore_index=True)

#add at the end a column for the date i downloaded it.
df1['download_date'] = pd.to_datetime('2024-05-21')

In [ ]:
#import second set
current_2 = pd.read_csv('2 active leads.csv')
dead_2 = pd.read_csv('2 dead Leads.csv')
warm_2 = pd.read_csv('2 warm leads.csv')

#cobine them
df2 = pd.concat([current_2, dead_2, warm_2], ignore_index=True)

#add at the end a column for the date i downloaded it.
df2['download_date'] = pd.to_datetime('2024-05-26')

In [ ]:
#import third set
current_3 = pd.read_csv('curent_lead_3.csv')
dead_3 = pd.read_csv('dead_lead3.csv')
warm_3 = pd.read_csv('warm_lead3.csv')

#cobine them
df3 = pd.concat([current_3, dead_3, warm_3], ignore_index=True)

#add at the end a column for the date i downloaded it.
df3['download_date'] = pd.to_datetime('2024-06-21')

In [ ]:
#import fourth set
current_4 = pd.read_csv('Active leads 4.csv')
dead_4 = pd.read_csv('dead leads 4.csv')
warm_4 = pd.read_csv('warm leads 4.csv')

#cobine them
df4 = pd.concat([current_3, dead_3, warm_3], ignore_index=True)

#add at the end a column for the date i downloaded it.
df4['download_date'] = pd.to_datetime('2024-06-24')

In [ ]:
# combine all
df = pd.concat([df1, df2, df3, df4], ignore_index=True)

In [ ]:
# show all empty columns
df.columns[df.isnull().all()]

Index(['Property Street Address 2', 'Mortgage Date', 'Tax Assessed Year',
       'Is There a Garage?', 'Garage Size', 'Assignment Contract Date',
       'Buyer Name', 'Buyer Phone Number', 'Buyer Email', 'Accountant',
       'Admin', 'Bookkeeper', 'Closing Coordinator', 'Co-Owner',
       'Lead Manager', 'Marketing Assistant', 'Marketing Manager',
       'Office Manager', 'Other Role', 'Project Manager', 'Property Analyst',
       'Property Manager', 'Transaction Coordinator', 'Real Estate Agent'],
      dtype='object')

In [ ]:
#drop all empty and unnecessary columns
to_drop = ['Property Street Address 2', 'Is There a Garage?', 'Garage Size','Garage attached or detached','Appointment Date',
           'Offer Price', 'Offer Date', 'Under Contract Date','Under Contract Price', 'Schedule Closing Date',
           'Expected Profit', 'Assignment Contract Date', 'Buyer Name', 'Buyer Phone Number','Buyer Email', 'Tags',
           'Dead Lead Reason', 'Accountant', 'Acquisition Manager', 'Acquisition Sales Manager', 'Admin', 'Bookkeeper',
           'Closing Coordinator', 'Co-Owner', 'Disposition Manager', 'Lead Manager', 'Marketing Assistant',
           'Marketing Manager','Office Manager', 'Other Role', 'Owner', 'Project Manager', 'Property Analyst',
           'Property Manager', 'Transaction Coordinator','Real Estate Agent', 'Mortgage Date', 'Tax Assessed Year']


df.drop(to_drop, axis=1, inplace=True)

In [ ]:
#count duplicates
df.duplicated().sum()
#drop duplicates
df = df.drop_duplicates(keep='first')

In [ ]:
#create a new column for phone number area code
df['Phone Area Code'] = df['Phone Number'].str.extract(r'\((\d{3})\)')

<ipython-input-10-df4f7fdff70b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Phone Area Code'] = df['Phone Number'].str.extract(r'\((\d{3})\)')


In [ ]:
#identify how many duplicate properties there are in the dataset based on the phone numbers.
phone_counts = df['Phone Number'].value_counts()

# Calculate the sum of duplicates by summing counts greater than 1 and subtracting 1 for each duplicate
sum_duplicates = (phone_counts[phone_counts > 1] - 1).sum()

# Display the sum of all duplicates
print(sum_duplicates)

5078


In [ ]:
#If a lead change comes up later as a worser lead, keep only the better one, and vice versa

#create a column with a score for lead status
lead_to_score = {'New Leads': -1,'Dead Lead' : 0, 'Warm Lead': 1, 'No Contact Made': 2, 'Contact Made' : 3,
                 'Hot Leads🔥': 4, 'Appointments Set' : 5, 'Offers Made' : 6, 'Under Contract' : 7}

# Create the 'score' column by mapping the 'rating' column
df['score'] = df['Lead Status'].map(lead_to_score)

In [ ]:
#before dopping duplicatse:
df['Lead Status'].value_counts()

Lead Status
Dead Lead           3194
Warm Lead           2036
Contact Made         955
No Contact Made      270
Offers Made          104
Hot Leads🔥            84
Appointments Set      70
New Leads             69
Under Contract        30
Name: count, dtype: int64

In [ ]:
# Sort the DataFrame by 'phone number' and 'score' in descending order
df = df.sort_values(by=['Phone Number', 'score'], ascending=[True, False])

# Drop duplicates of phone numbers, keeping the first occurrence (which has the highest score due to sorting)
df = df.drop_duplicates(subset='Phone Number', keep='first')

# Reset index for a clean DataFrame
df = df.reset_index(drop=True)

In [ ]:
#after dropping duplicates
df['Lead Status'].value_counts()

Lead Status
Dead Lead           752
Warm Lead           451
Contact Made        318
No Contact Made      69
Offers Made          54
Hot Leads🔥           37
Appointments Set     24
New Leads            20
Under Contract        9
Name: count, dtype: int64

In [ ]:
# Calculate the percentage of each unique value in the ''Lead Status'' column
percentage_counts = df['Lead Status'].value_counts(normalize=True) * 100
print(percentage_counts)

Lead Status
Dead Lead           43.367935
Warm Lead           26.009227
Contact Made        18.339100
No Contact Made      3.979239
Offers Made          3.114187
Hot Leads🔥           2.133795
Appointments Set     1.384083
New Leads            1.153403
Under Contract       0.519031
Name: proportion, dtype: float64


In [ ]:
lead_to_quality = {'New Leads': 'Ok','Dead Lead' : 'Bad', 'Warm Lead': 'Bad', 'No Contact Made': 'Ok', 'Contact Made' : 'Ok',
                 'Hot Leads🔥': 'Good', 'Appointments Set' : 'Good', 'Offers Made' : 'Good', 'Under Contract' : 'Good'}

# Create the 'score' column by mapping the 'rating' column
df['Lead Quality'] = df['Lead Status'].map(lead_to_quality)

In [ ]:
percentage_q_counts = df['Lead Quality'].value_counts(normalize=True) * 100
print(percentage_q_counts)

Lead Quality
Bad     69.377163
Ok      23.471742
Good     7.151096
Name: proportion, dtype: float64


In [ ]:
column_order = ['First Name', 'Last Name', 'Phone Area Code', 'Phone Number', 'Email Address',
       'Lead Status', 'score', 'Lead Quality', 'Lead Source', 'Campaign Name',
       'Property Street Address', 'Property City', 'Property State',
       'Property Zip', 'Mailing Address','Bedroom', 'Bathroom', 'Apporx Sqft',
       'Lot Size Sqft', 'Year Buit', 'House Type', 'Mortgage Amount',
       'Tax Assessed Value', 'Tax Billed Amount', 'Last Sold Price',
       'Prior Sale Price', 'Lead Created Date', 'download_date']

# Reorder columns
df = df[column_order]

In [ ]:
df.columns

Index(['First Name', 'Last Name', 'Phone Area Code', 'Phone Number',
       'Email Address', 'Lead Status', 'score', 'Lead Quality', 'Lead Source',
       'Campaign Name', 'Property Street Address', 'Property City',
       'Property State', 'Property Zip', 'Mailing Address', 'Bedroom',
       'Bathroom', 'Apporx Sqft', 'Lot Size Sqft', 'Year Buit', 'House Type',
       'Mortgage Amount', 'Tax Assessed Value', 'Tax Billed Amount',
       'Last Sold Price', 'Prior Sale Price', 'Lead Created Date',
       'download_date'],
      dtype='object')

In [ ]:
df.loc[df['Phone Area Code'].isnull()]

,First Name,Last Name,Phone Area Code,Phone Number,Email Address,Lead Status,score,Lead Quality,Lead Source,Campaign Name,...,Lot Size Sqft,Year Buit,House Type,Mortgage Amount,Tax Assessed Value,Tax Billed Amount,Last Sold Price,Prior Sale Price,Lead Created Date,download_date
1732,NaN,NaN,NaN,+266696687,NaN,Dead Lead,0,Bad,Cold Calling,Cold Callers,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-04-10,2024-05-21
1733,NaN,NaN,NaN,266696687,NaN,Dead Lead,0,Bad,Cold Calling,Cold Callers,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4/10/24,2024-05-26


In [ ]:
df.isnull().sum()

First Name                   76
Last Name                   170
Phone Area Code               2
Phone Number                  0
Email Address              1579
Lead Status                   0
score                         0
Lead Quality                  0
Lead Source                   0
Campaign Name                92
Property Street Address     117
Property City               119
Property State              121
Property Zip                227
Mailing Address             536
Bedroom                     925
Bathroom                    747
Apporx Sqft                 547
Lot Size Sqft               611
Year Buit                   548
House Type                  529
Mortgage Amount            1147
Tax Assessed Value          536
Tax Billed Amount           566
Last Sold Price             934
Prior Sale Price           1061
Lead Created Date             0
download_date                 0
dtype: int64

In [ ]:
#download df to computer
df.to_csv('clean_df.csv', index=False)